## Reading/Writing Parquet Files From/to OCI Object Storage with Pandas

The setup for this notebook is simple. I used the `generalml_p37_cpu_v1` conda environment. This environment has ADS,pyarrow, pandas, snappy, and fastparquet pre-installed. 

I also upgraded the version of ADS. In this notebook ADS version 2.5.8 was used. 

To read/write files to Object Storage, use `ocifs` : 
* github.com/oracle/ocifs
* docs: https://docs.oracle.com/en-us/iaas/tools/ocifs-sdk/latest/unix-operations.html 

In [ ]:
#!pip install oracle-ads --upgrade

In [ ]:
import ads 
from ads.common.auth import default_signer
import pandas as pd 
import fsspec
from ocifs import OCIFileSystem

# Using resource principal auth method: 
print(ads.__version__)
ads.set_auth(auth="resource_principal")

In [ ]:
# object storage bucket + data 
# this bucket is publicly available 
bucket = "hosted-ds-datasets"
namespace = "bigdatadatasciencelarge"

# Single Large File

In [ ]:
# ~ 440 MB, 14M rows. 
large_files = ["nyc_tlc/2009/01/data.parquet"] # NYC Taxi dataset 

In [ ]:
# using the `pyarrow` engine. You can also use `fastparquet`. 
for f in large_files: 
    df = pd.read_parquet(f"oci://{bucket}@{namespace}/{f}", 
                     storage_options=default_signer(),
                     engine="pyarrow")
    print(f"file {f}")
    print(df.head())
    print(f"size {df.shape}")

# Multiple Large Files

In [ ]:
from oci.auth.signers import get_resource_principals_signer

In [ ]:
# Using resource principal for authn. 
fs = OCIFileSystem(signer=get_resource_principals_signer())

In [ ]:
# This is a small sample of the overall NYC Taxi dataset. There are 4 files in total, each of size ~ 450MB. 
relevant_files = fs.ls(f"{bucket}@{namespace}/nyc_tlc/2009/")

In [ ]:
%%time 
# this operation takes about 50 secs: 
df = pd.concat((pd.read_parquet(f"oci://{f}/data.parquet", storage_options=default_signer(), engine="pyarrow") for f in relevant_files), 
               ignore_index=True)

In [ ]:
# ~ 56M rows 
df.shape

## Write Parquet Files to Object Storage: 

In [ ]:
%%time 

# Writing the dataframe as a parquet file to Object Storage. 
# Insert the name of your bucket and namespace you want to write parquet. 
# this operation takes about 2 mins
your_bucket = ""
your_namespace = ""

df.to_parquet(f"oci://{your_bucket}@{your_namespace}/taxi-data.parquet", 
                     storage_options=default_signer())